In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
df = pd.read_csv('/content/Data Scrapping (before preprocessing) FIKS - Dataset - Data Scrapping (before preprocessing).xlsx - Dataset.csv')
df.head()

,username,text,created_at,sumber
0,anggi toXinc,Rungkad mulu di crypto karna fomo ðŸ˜­ðŸ˜­ ken...,Wed Jan 29 14:49:37 +0000 2025,X
1,Sinau â‚¿itcoin ðŸ§¡,wis menang lur .. kok isih do denial sih.. b...,Wed Jan 29 12:24:13 +0000 2025,X
2,MunchMunch.base.eth ðŸ’™,Yang gw liat dari kemarin chart beberapa koin ...,Wed Jan 29 12:09:50 +0000 2025,X
3,FLOKI,saham us dump parah yakali market crypto bakal...,Wed Jan 29 10:52:13 +0000 2025,X
4,MICIN COIN ðŸ‡®ðŸ‡©,"Jangan FOMO, Cek Dulu! ðŸš€ Koin naik gila-...",Wed Jan 29 08:46:25 +0000 2025,X


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9528 entries, 0 to 9527
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   username    9525 non-null   object
 1   text        9527 non-null   object
 2   created_at  9528 non-null   object
 3   sumber      9528 non-null   object
dtypes: object(4)
memory usage: 297.9+ KB


In [ ]:
NORM_DICT = {
    "btc": "bitcoin", "bitcoin": "bitcoin", "bitc": "bitcoin", "btcin": "bitcoin",
    "eth": "ethereum", "etherium": "ethereum", "etherum": "ethereum", "etereum": "ethereum",
    "bnb": "binance coin", "binance": "binance", "binanc": "binance", "benance": "binance",
    "xrp": "ripple", "ripel": "ripple",
    "sol": "solana", "solan": "solana",
    "doge": "dogecoin", "dog": "dogecoin",
    "shib": "shiba inu", "shiba": "shiba inu",
    "usdt": "tether", "ustd": "tether", # Typo umum
    "alt": "altcoin", "alts": "altcoin", "altcoins": "altcoin",
    "crypto": "kripto", "cripto": "kripto", "krypto": "kripto", "cryto": "kripto", "crip": "kripto",
    "indodax": "indodax", "indodak": "indodax", "indodex": "indodax",
    "tokocrypto": "tokocrypto", "tokocripto": "tokocrypto", "toko": "tokocrypto",
    "binance": "binance", "binan": "binance",
    "opensea": "opensea", "open sea": "opensea",
    "wallet": "dompet digital", "walet": "dompet digital", "wallett": "dompet digital",

    "cuan": "untung", "cuann": "untung", "cwan": "untung", "profit": "untung", "profitt": "untung",
    "boncos": "rugi besar", "boncos": "rugi besar", "bancos": "rugi besar",
    "loss": "rugi", "los": "rugi", "loos": "rugi", "lose": "rugi",
    "sangkut": "terjebak rugi", "nyangkut": "terjebak rugi", "sangkot": "terjebak rugi", "sangkutt": "terjebak rugi",
    "serok": "beli harga bawah", "srok": "beli harga bawah", "serock": "beli harga bawah",
    "entry": "masuk beli", "entri": "masuk beli",
    "tp": "ambil untung", "take profit": "ambil untung",
    "cl": "jual rugi", "cutloss": "jual rugi", "cut loss": "jual rugi", "catlos": "jual rugi",
    "hold": "tahan", "holld": "tahan", "tahan": "tahan",
    "hodl": "tahan aset", "hode": "tahan aset",
    "fud": "berita negatif palsu", "fudder": "penyebar berita negatif",
    "fomo": "takut ketinggalan", "fomu": "takut ketinggalan",
    "dyor": "riset sendiri",
    "bull": "tren naik", "bullish": "tren naik", "bulish": "tren naik", "bulis": "tren naik",
    "bear": "tren turun", "bearish": "tren turun", "bearis": "tren turun", "berish": "tren turun",
    "scam": "penipuan", "skam": "penipuan", "scammer": "penipu",
    "rugpull": "penipuan kabur", "rug": "penipuan kabur", "rugpul": "penipuan kabur",
    "to the moon": "harga naik tinggi", "tothemoon": "harga naik tinggi", "moon": "harga naik tinggi",
    "pump": "harga naik", "pompom": "hasutan beli",
    "dump": "harga turun", "dam": "harga turun",
    "anjlok": "turun drastis", "anjloq": "turun drastis",
    "long": "posisi beli",
    "short": "posisi jual", "sort": "posisi jual",
    "liquid": "likuidasi habis", "likuid": "likuidasi habis", "likuidasi": "likuidasi habis", "mc": "modal habis",
    "wd": "tarik dana", "withdraw": "tarik dana", "witdraw": "tarik dana",
    "depo": "deposit", "deposite": "deposit",
    "gas": "biaya transaksi", "gasfee": "biaya transaksi", "fee": "biaya",

    # Negasi (PENTING untuk sentimen)
    "gak": "tidak", "ga": "tidak", "gk": "tidak", "tdk": "tidak", "nggak": "tidak",
    "enggak": "tidak", "kaga": "tidak", "kagak": "tidak", "nda": "tidak", "ndak": "tidak",
    "ora": "tidak", "tak": "tidak", "g": "tidak", "ngak": "tidak", "ngk": "tidak",

    # Kata Ganti Orang
    "sy": "saya", "gw": "saya", "gue": "saya", "gwe": "saya", "gua": "saya",
    "w": "saya", "aqu": "saya", "aku": "saya", "ak": "saya", "q": "saya", "ane": "saya",
    "lu": "kamu", "loe": "kamu", "lo": "kamu", "u": "kamu", "kamu": "kamu", "km": "kamu",
    "ente": "kamu", "gan": "juragan", "agan": "juragan", "bang": "abang", "bg": "abang",
    "bng": "abang", "banh": "abang", "kak": "kakak", "bro": "saudara", "sis": "saudari",
    "min": "admin", "mimin": "admin",
    "mrk": "mereka", "mereka": "mereka",
    "kita": "kita", "kta": "kita",

    # Kata Sambung & Tanya
    "yg": "yang", "yng": "yang", "yang": "yang",
    "dgn": "dengan", "dengn": "dengan",
    "dr": "dari", "dri": "dari",
    "dlm": "dalam", "dalem": "dalam",
    "kpn": "kapan", "kapan": "kapan",
    "knp": "kenapa", "napa": "kenapa", "np": "kenapa", "knapa": "kenapa",
    "dmn": "dimana", "dimana": "dimana",
    "gmn": "bagaimana", "gimana": "bagaimana", "gmna": "bagaimana",
    "utk": "untuk", "untk": "untuk", "tuk": "untuk",
    "sm": "sama", "ama": "sama",
    "krn": "karena", "karna": "karena", "kerna": "karena",
    "kl": "kalau", "kalo": "kalau", "klo": "kalau", "jika": "jika",
    "tp": "tapi", "tpi": "tapi", "tap": "tapi",
    "jd": "jadi", "jdi": "jadi",
    "bkn": "bukan", "bukn": "bukan",
    "ad": "ada", "ada": "ada", "adh": "ada",
    "udh": "sudah", "sdh": "sudah", "dah": "sudah", "uda": "sudah", "wes": "sudah",
    "blm": "belum", "blom": "belum", "lom": "belum",
    "ni": "ini", "ini": "ini", "ne": "ini",
    "tu": "itu", "itu": "itu", "tuh": "itu",
    "kok": "kenapa", "koq": "kenapa", "ko": "kenapa",

    # Kata Kerja & Sifat
    "bgt": "banget", "banget": "banget", "bngt": "banget", "beud": "banget", "bet": "banget", "kali": "banget",
    "tau": "tahu", "taw": "tahu",
    "mau": "ingin", "mo": "ingin", "pengen": "ingin", "pengin": "ingin",
    "bisa": "dapat", "bs": "dapat", "iso": "dapat",
    "dpt": "dapat", "dapet": "dapat",
    "bljr": "belajar", "belajar": "belajar",
    "pke": "pakai", "pake": "pakai",
    "liat": "lihat", "lht": "lihat",
    "blg": "bilang", "bilang": "bilang",
    "ngomong": "bicara",
    "bener": "benar", "bnr": "benar", "betul": "benar", "tul": "benar",
    "salah": "salah", "slh": "salah",
    "bagus": "bagus", "bgs": "bagus", "mantap": "bagus", "mantul": "bagus", "ok": "bagus", "oke": "bagus",
    "jelek": "buruk", "jlk": "buruk", "buruk": "buruk", "parah": "buruk",
    "aja": "saja", "aj": "saja", "ae": "saja", "doang": "saja",
    "jg": "juga", "jga": "juga", "jugak": "juga",
    "gini": "begini", "gni": "begini",
    "gitu": "begitu", "gtu": "begitu",
    "emang": "memang", "emg": "memang", "mang": "memang",
    "mungkin": "mungkin", "mgkn": "mungkin", "mngkin": "mungkin",
    "siap": "siap", "saip": "siap", # Typo
    "pasti": "pasti", "pst": "pasti",
    "moga": "semoga", "smoga": "semoga", "mdh2an": "semoga",
    "aamiin": "amin", "amin": "amin",
    "thx": "terima kasih", "tq": "terima kasih", "makasih": "terima kasih", "mksh": "terima kasih",
    "wkwk": "tertawa", "wkwkwk": "tertawa", "hahaha": "tertawa", "wk": "tertawa",
    "hiks": "sedih", "nangis": "sedih",
    "anjay": "wow", "anjir": "wow", "gila": "wow", "gokil": "wow", # Bisa positif/negatif tergantung konteks
}

In [ ]:
!pip install emoji
import pandas as pd
import re
import emoji


def preprocess_pipeline(text):
    text = str(text) if pd.notna(text) else ""

    text = emoji.demojize(text, delimiters=(" ", " "))

    # 3. Lowercase
    text = text.lower()

    text = re.sub(r'http\S+|www\S+|https\S+|@\S+|\d+|#|[^a-zA-Z\s]', ' ', text)

    words = text.split()
    text = ' '.join([NORM_DICT.get(word, word) for word in words])

    return text

df['text'] = df['text'].apply(preprocess_pipeline)

df.drop_duplicates(subset=['text'], inplace=True)

df = df[df['text'].str.split().str.len().between(3, 35)]

print(f"Sisa data: {len(df)}")
print(df['text'].head())

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 12.9 MB/s eta 0:00:00
Sisa data: 7878
0    rungkad mulu di kripto karena takut ketinggala...
1    wis menang lur kenapa isih do denial sih bitco...
2    yang saya lihat dari kemarin chart beberapa ko...
3    saham us harga turun buruk yakali market kript...
5    ada apa dengan market kripto lesu banget nge t...
Name: text, dtype: object


remove emoji


In [ ]:
len(df)

7878

In [ ]:
# df['text'].iloc[1]

In [ ]:
df.to_csv("Data_Crypto_AFTER_PREPROCESSING.csv")
print("Data_Crypto_AFTER_PREPROCESSING.csv sudah tersimpan")

Data_Crypto_AFTER_PREPROCESSING.csv sudah tersimpan
